In [1]:

from tabulate import tabulate
import pandas as pd
   #
pd.set_option("display.width", 480)

print("--")


--


In [2]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [3]:
NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [4]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)

<_Graph my_graph, 9w7tCeQ52o4A2SBjyfmbcKo4Apha3wYvz55WHigpPQDV, 2>


In [5]:
l_result1 = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )
# l_result1.view()
print(tabulate(l_result1, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+-----------------+-------------+-----------+------------------+--------+----------+
|    |   n.internal_id | n.labels    | n.LABEL   | n.airport_name   | n.id   | n.type   |
|----+-----------------+-------------+-----------+------------------+--------+----------|
|  0 |               1 | ['Airport'] | Airport   | Denver           | DEN    | node     |
|  1 |               0 | ['Airport'] | Airport   | San Jose         | SJC    | node     |
|  2 | 281474976710656 | ['Airport'] | Airport   | Milwaukee        | MKE    | node     |
|  3 | 562949953421312 | ['Airport'] | Airport   | Chicago O-Hare   | ORD    | node     |
+----+-----------------+-------------+-----------+------------------+--------+----------+


In [6]:

#  graph.project() is used to create a homogenous group, from a heterogeneous one,
#  necessary to support some analytics
#
#  Here we'll add stuff to the graph that analytics.sssp() will not like ..


l_query = """
   CREATE ( n: Restaurant { restaurant_code: 'PAP' } )               //  Notice single curly braces
   SET n.restaurant_name = 'Pappadeauxs Seafood Kitchen' 
   """
      #
display(l_result1 = my_graph.query(l_query))


          0/? [?op/s]

In [7]:
l_result1 = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )
# l_result1.view()
print(tabulate(l_result1, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+-----------------+----------------+-----------+------------------+--------+----------+---------------------+-----------------------------+
|    |   n.internal_id | n.labels       | n.LABEL   | n.airport_name   | n.id   | n.type   | n.restaurant_code   | n.restaurant_name           |
|----+-----------------+----------------+-----------+------------------+--------+----------+---------------------+-----------------------------|
|  0 |               0 | ['Airport']    | Airport   | San Jose         | SJC    | node     | nan                 | nan                         |
|  1 |               1 | ['Airport']    | Airport   | Denver           | DEN    | node     | nan                 | nan                         |
|  2 | 281474976710656 | ['Airport']    | Airport   | Milwaukee        | MKE    | node     | nan                 | nan                         |
|  3 | 562949953421313 | ['Restaurant'] | nan       | nan              | nan    | node     | PAP                 | Pappadeauxs Sea